# Explore Runtimes:

In [1]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis/BXD"

In [2]:
cd("..")

In [3]:
include("../test/BXDdata_for_test.jl");

In [4]:
include("../src/parallel_helpers.jl");

In [5]:
using Plots

In [6]:
size(pheno)

(79, 35556)

In [7]:
size(pheno_y)

(79, 1)

In [8]:
size(geno)

(79, 7321)

In [9]:
size(kinship)

(79, 79)

In [10]:
(n, m, p)

(79, 35556, 7321)

In [11]:
Threads.nthreads()

16

In [12]:
using LinearAlgebra

In [13]:
BLAS.get_num_threads()

16

In [52]:
full_results_ordinary = Array{Float64, 2}(undef, p, m);

In [ ]:
@time begin
for i in 1:m
    pheno_yi = reshape(pheno[:, i], :, 1);
    full_results_ordinary[:, i] = scan(pheno_yi, geno, kinship; method = "null", reml = true)[3]
end
end

In [26]:
full_results_ordinary

7321×10 Matrix{Float64}:
 0.000120087  0.0459408  0.00296438  …  0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438  …  0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438     0.140737    0.0077483    0.0348749
 0.000120087  0.0459408  0.00296438  …  0.140737    0.0077483    0.0348749
 0.000120007  0.0459415  0.00296468     0.140736    0.00774795   0.0348743
 0.00953337   0.0651315  0.0179876      0.0930242   0.000555067  0.0162758


In [42]:
(Y0, X0, lambda0) = transform_rotation(pheno, geno, kinship);

In [43]:
X0_intercept = reshape(X0[:, 1], :, 1);
X0_covar = X0[:, 2:end];

In [44]:
# X00 = resid(X0[:, 2:end], X0_intercept);

In [45]:
n

79

In [46]:
function r2lod(r::Float64)
    rsq = (r/n)^2
    return -(n/2.0) * log10(1.0-rsq);
end

r2lod (generic function with 1 method)

In [47]:
r2lod.([0.1, 0.9])

2-element Vector{Float64}:
 2.748701454592006e-5
 0.0022265908888506967

In [53]:
full_results_lite= Array{Float64, 2}(undef, p, m);

In [54]:
@time begin
for i in 1:m
    pheno_y0i = reshape(Y0[:, i], :, 1);
    vc = fitlmm(pheno_y0i, X0_intercept, lambda0; reml = true);
    sqrtw = sqrt.(makeweights(vc.h2, lambda0));
    
    y00i = rowMultiply(pheno_y0i, sqrtw);
    X00_intercept = rowMultiply(X0_intercept, sqrtw);
    X00_covar = rowMultiply(X0_covar, sqrtw);
    
    y00 = resid(y00i, X00_intercept);
    X00 = resid(X00_covar, X00_intercept);
        
    sy = std(y00, corrected = false, dims = 1) |> vec;
    sx = std(X00, corrected = false, dims = 1) |> vec;
    colDivide!(y00, sy);
    colDivide!(X00, sx);
    R = y00' * X00;
    
    full_results_lite[:, i] = r2lod.(R);
    
end

end

538.697023 seconds (2.12 G allocations: 515.288 GiB, 4.09% gc time)


In [55]:
full_results_lite[1:6, :]

6×35556 Matrix{Float64}:
 0.000121349  0.046365  0.00300596  …  0.0470942  0.00338421  0.352502
 0.000121349  0.046365  0.00300596     0.0470942  0.00338421  0.352502
 0.000121349  0.046365  0.00300596     0.0470942  0.00338421  0.352502
 0.000121349  0.046365  0.00300596     0.0470942  0.00338421  0.352502
 0.000121349  0.046365  0.00300596     0.0470942  0.00338421  0.352502
 0.000121349  0.046365  0.00300596  …  0.0470942  0.00338421  0.352502

In [ ]:
full_results_ordinary[1:6, :]

In [ ]:
maxSqDiff(full_results_lite, full_results_ordinary)

## Estimate heritability independently for every trait:

### Iteratively:

In [ ]:
function get_weights_for_timing(y0::Array{Float64, 2}, X0::Array{Float64, 2}, lambda0::Array{Float64, 1};
                   reml::Bool = false)
    
    # fit null lmm
    out00 = fitlmm(y0, reshape(X0[:, 1], :, 1), lambda0; reml = reml)
    # weights proportional to the variances
    sqrtw = sqrt.(makeweights(out00.h2, lambda0));
    
    return sqrtw
    
end

function get_h2s_for_timing(y0::Array{Float64, 2}, X0::Array{Float64, 2}, lambda0::Array{Float64, 1};
                   reml::Bool = false)
    
    # fit null lmm
    out00 = fitlmm(y0, reshape(X0[:, 1], :, 1), lambda0; reml = reml)
    # weights proportional to the variances
    # sqrtw = sqrt.(makeweights(out00.h2, lambda0));
    
    # return sqrtw
    return out00.h2
    
end

In [ ]:
m

In [ ]:
nprocs()

In [ ]:
Threads.nthreads()

In [ ]:
(Y0, X0, lambda0) = transform_rotation(pheno, geno, kinship); # rotate the entire matrix of all traits

In [ ]:
h2s_list = Array{Float64, 1}(undef, m);
weights_list = Array{Float64, 2}(undef, n, m);

In [ ]:
@time begin 
    (Y0, X0, lambda0) = transform_rotation(pheno, geno, kinship); # rotate the entire matrix of all traits
    
    for k in 1:m
        y0_k = reshape(Y0[:, k], :, 1);
        h2s_list[k] = get_h2s_for_timing(y0_k, X0, lambda0; reml = true);
    end
    
end

In [ ]:
@time begin
    
    (Y0, X0, lambda0) = transform_rotation(pheno, geno, kinship); # rotate the entire matrix of all traits
    
    for k in 1:m
        y0_k = reshape(Y0[:, k], :, 1);
        weights_list[:, k] = get_weights_for_timing(y0_k, X0, lambda0; reml = true);
    end
    
end

In [ ]:
get_weights_for_timing(reshape(Y0[:, 7919], :, 1), X0, lambda0; reml = true)

In [ ]:
histogram(h2s_list, legend = false)

In [ ]:
p

In [ ]:
lod = Array{Float64, 2}(undef, p, m);

In [ ]:
BLAS.get_num_threads()

In [ ]:
function threads_by_blocks(r0perm::Array{Float64, 2}, X00::Array{Float64, 2}, nblocks::Int64)
    # Does distributed processes of calculations of LOD scores for markers in each block

    p = size(X00, 2);

    ## (Create blocks...)
    
    block_size = ceil(Int, p/nblocks);
    blocks = createBlocks(p, block_size);

    LODs_blocks = tmap(x -> calcLODs_block(r0perm, X00, x), 16, blocks);
    results = reduce(hcat, LODs_blocks);

    return results

end

In [ ]:
function scan_perms_threadsBlocks(y::Array{Float64,2}, g::Array{Float64,2}, K::Array{Float64,2};
                                reml::Bool = false,
                                nperms::Int64 = 1024, rndseed::Int64 = 0, original::Bool = true,
                                # (options for blocks, nperms distribution methods...)
                                option::String = "by blocks", nblocks::Int64 = 1, ncopies::Int64 = 1, 
                                nprocs::Int64 = 0)

    (y0, X0, lambda0) = transform_rotation(y, g, K); # rotation of data
    (r0, X00) = transform_reweight(y0, X0, lambda0; reml = reml); # reweighting and taking residuals
    r0perm = transform_permute(r0; nperms = nperms, rndseed = rndseed, original = original);

    if option == "by blocks"
        results = threads_by_blocks(r0perm, X00, nblocks);
    elseif option == "by nperms"
        results = distribute_by_nperms(r0, X00, nperms, ncopies, original);
    else
        throw(error("Option unsupported."))
    end

    return results

end

In [ ]:
block_bounds = quantile(h2s_list, [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])

In [ ]:
length(block_bounds)/2

In [ ]:
sum(map(x -> (x < block_bounds[5] && x > block_bounds[4]), h2s_list))

### Parellely:

#### Distributed-processes

In [ ]:
using Distributed

In [ ]:
@everywhere begin
    
    include("../test/BXDdata_for_test.jl");
    include("../src/parallel_helpers.jl");
    
    function get_h2s_for_timing(y0::Array{Float64, 2}, X0::Array{Float64, 2}, lambda0::Array{Float64, 1};
                   reml::Bool = false)
    
    # fit null lmm
    out00 = fitlmm(y0, reshape(X0[:, 1], :, 1), lambda0; reml = reml)
    # weights proportional to the variances
    # sqrtw = sqrt.(makeweights(out00.h2, lambda0));
    
    # return sqrtw
    return out00.h2
    
    end
end

In [ ]:
@time begin 
    (Y0, X0, lambda0) = transform_rotation(pheno, geno, kinship); # rotate the entire matrix of all traits
    
    pmap_h2s = pmap(x -> get_h2s_for_timing(reshape(Y0[:, x], :, 1), X0, lambda0; reml = true), 1:m)
end

In [ ]:
Threads.nthreads()